## Statistics of Dataset

### Subtask B

In [7]:
import pandas as pd
df = pd.read_json("./data/SubtaskB_split.jsonl", lines=True)
print(len(df))

122811


In [9]:
df.source.value_counts()

source
wikihow      23556
reddit       20999
outfox       20999
arxiv        20998
wikipedia    19368
peerread     16891
Name: count, dtype: int64

In [10]:
df.model.value_counts()

model
bloomz           17332
human            17179
chatGPT          16892
cohere           16678
gpt4             14344
davinci          14340
dolly            14046
gpt-3.5-turbo     6000
davinci-003       3000
dolly-v2-12b      3000
Name: count, dtype: int64

### Subtask A

In [12]:
df = pd.read_json("./data/SubtaskA.jsonl", lines=True)

In [17]:
# print(df.source.unique(), df.model.unique())
models = ['human', 'davinci', 'chatGPT', 'cohere', 'dolly', 'bloomz', 'gpt4']
domains = ['wikipedia', 'wikihow', 'reddit', 'arxiv', 'peerread']
print(models)
print(domains)

['human', 'davinci', 'chatGPT', 'cohere', 'dolly', 'bloomz', 'gpt4']
['wikipedia', 'wikihow', 'reddit', 'arxiv', 'peerread']


In [18]:
for label in [True, False]:
    data = df[df["parallel"] == label]
    print(len(data))
    table = {}
    for domain in domains:
        temp = data[data["source"] == domain]
        # print(f"{domain}: {len(temp)}")
        table[domain] = {}
        for model in models:
            t = temp[temp["model"] == model]
            # print(f"{model}: {len(t)}")
            table[domain][model] = len(t)
        table[domain]["Total machine"] = len(temp) - len(temp[temp["model"] == 'human']) - len(temp[temp["model"] == 'gpt4']) 
    table = pd.DataFrame(table).T
    # table
    print(table.round(2).astype(str).to_latex())

101812
\begin{tabular}{lllllllll}
\toprule
 & human & davinci & chatGPT & cohere & dolly & bloomz & gpt4 & Total machine \\
\midrule
wikipedia & 2336 & 3000 & 2995 & 2336 & 2702 & 2999 & 3000 & 14032 \\
wikihow & 2999 & 3000 & 5557 & 3000 & 3000 & 3000 & 3000 & 17557 \\
reddit & 3000 & 3000 & 3000 & 3000 & 3000 & 2999 & 3000 & 14999 \\
arxiv & 2998 & 3000 & 3000 & 3000 & 3000 & 3000 & 3000 & 15000 \\
peerread & 2847 & 2340 & 2340 & 2342 & 2344 & 2334 & 2344 & 11700 \\
\bottomrule
\end{tabular}

50997
\begin{tabular}{lllllllll}
\toprule
 & human & davinci & chatGPT & cohere & dolly & bloomz & gpt4 & Total machine \\
\midrule
wikipedia & 11997 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
wikihow & 13000 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
reddit & 13000 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
arxiv & 13000 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
peerread & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
\bottomrule
\end{tabular}



In [19]:
df.source.value_counts()

source
wikihow      36556
reddit       33999
arxiv        33998
wikipedia    31365
peerread     16891
Name: count, dtype: int64

In [20]:
df.model.value_counts()

model
human      65177
chatGPT    16892
gpt4       14344
davinci    14340
bloomz     14332
dolly      14046
cohere     13678
Name: count, dtype: int64

## Extract GLTR Features

In [21]:
import sys
sys.path.append("./src/GLTR/")
from src.detector.extract_gltr_features import get_gltr_feature_in_batch

Found API <class 'backend.api.LM'> with name gpt-2-small


### Subtask A Dataset

In [ ]:
import pandas as pd

data = pd.read_json('data/SubtaskA_split.jsonl', lines=True)
texts = list(data['text'])
print(len(texts))

features = get_gltr_feature_in_batch(text_list = texts, savepath="./data/gltr_features/features_subtaskA.json")

# features = dict(pd.read_json("./data/gltr_features/features_subtaskA.json", typ="series"))
assert(len(features) == len(data))
data["GLTR_features"] = list(features.values())
data.to_json("./data/gltr_features/SubtaskA_splits_GLTR.jsonl", lines=True, orient="records")

### Subtask B data

In [ ]:
import pandas as pd

data = pd.read_json('./data/SubtaskB_split.jsonl', lines=True)
texts = list(data['text'])
print(len(texts))

features = get_gltr_feature_in_batch(text_list = texts, savepath="./data/gltr_features/features_subtaskB.json")
# features = dict(pd.read_json("./data/gltr_features/features_subtaskB.json", typ="series"))
assert(len(features) == len(data))
data["GLTR_features"] = list(features.values())
data.to_json("./data/gltr_features/SubtaskB_splits_GLTR.jsonl", lines=True, orient="records")

## Train and Test

### Subtask A

In [ ]:
import pandas as pd
from src.detector.gltr_lr import train, test

data = pd.read_json('./data/gltr_features/SubtaskA_splits_GLTR.jsonl', lines=True)
# data.loc[9156] 
data = data.drop(9156) # data["GLTR_features"] = []

random_seeds = [0, 10, 30, 55, 75]
models = list(data.model.unique())
models.remove('human')  # in any settings, we need human data
models.append('no')
print(models)

results_to_save = []
for random_seed in random_seeds:
    for model in models:
        data_train = data[data[f'{model}_{random_seed}'] == 'train']
        data_valid = data[data[f'{model}_{random_seed}'] == 'valid']
        data_test = data[data[f'{model}_{random_seed}'] == 'test']
        # print(len(data_train), len(data_valid), len(data_test))

        # get feature and labels
        X_train, y_train = get_gltr_features(data_train)
        X_valid, y_valid = get_gltr_features(data_valid)
        X_test, y_test = get_gltr_features(data_test)
        
        # train
        clf = train(X_train, y_train)

        # test
        # metrics = eval_binary(clf, X_test, y_test, pos_label=1, average="binary")
        results = test(X_test, y_test, clf)
        # save results of classification report to 'classification_report_results.jsonl'
        res = {
            'random_seed': random_seed,
            'model': model,
            'iteration': clf.n_iter_,
            'results': results
        }
        print(results["accuracy"])
        results_to_save.append(res)
    pd.Series(results_to_save).to_json("test_results/gltr_subtaskA.json")

pd.Series(results_to_save).to_json("test_results/gltr_subtaskA.json")
# results_to_save = list(pd.read_json("test_results/gltr_subtaskA.json", typ="series"))

### Subtask B

In [1]:
# Subtask B is a multiclass classification task, we need to decide classifer we want to use in sklear:
# one-vs-one: svc
# one-vs-rest: lr
import pandas as pd
from src.detector.gltr_svm import train, test, get_gltr_features

data = pd.read_json('data/gltr_features/SubtaskB_splits_GLTR.jsonl', lines=True)
data = data.drop(9156)
random_seeds = [0, 10, 30, 55, 75]

domains = list(data.source.unique())
domains.append('no')
print(domains)

results_to_save = []
for random_seed in random_seeds:
    for domain in domains:
        data_train = data[data[f'{domain}_{random_seed}'] == 'train']
        data_valid = data[data[f'{domain}_{random_seed}'] == 'valid']
        data_test = data[data[f'{domain}_{random_seed}'] == 'test']

        
        # get feature and labels
        X_train, y_train = get_gltr_features(data_train)
        X_valid, y_valid = get_gltr_features(data_valid)
        X_test, y_test = get_gltr_features(data_test)
        
        # train
        clf, svc_dfs = train(X_train, y_train)

        # test
        results_ovr = test(X_test, y_test, clf)
        results_ovo = test(X_test, y_test, svc_dfs)
        # save results of classification report to 'classification_report_results.jsonl'
        res = {
            'random_seed': random_seed,
            'domain': domain,
            'ovr_iter': clf.n_iter_,
            'results_ovr': results_ovr,
            'ovo_iter': svc_dfs.n_iter_,
            'results_ovo': results_ovo
        }
        print(results_ovr["accuracy"], results_ovo["accuracy"])
        results_to_save.append(res)
    pd.Series(results_to_save).to_json("test_results/subtaskB_0604.json")

pd.Series(results_to_save).to_json("test_results/subtaskB_0604.json")

/home/yuxiawang/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['arxiv', 'wikipedia', 'wikihow', 'reddit', 'peerread', 'outfox', 'no']
Trained [213 225 215 254 214 321 246] iterations.
Trained [ 4636 10341 12250  1686 12514 10828  3029  4724  4809  6236  2407  9694
  3189  7969  9260  2795 10051  9484  3578  1953  7363] iterations.
0.3447471187732165 0.323126012001143
Trained [197 246 163 233 190 302 221] iterations.
Trained [ 4894 10741 12336  2946 14072 15048  2770  6444  6728  6188  2734  9580
  3208  8305  9451  5015 10188  9363  5048  3422  7025] iterations.
0.35266174420405844 0.2981360045438116
Trained [251 276 203 258 232 278 186] iterations.
Trained [ 3710 10712 10994  2511  9848  9354  2949  5513  6154  5911  1672 10071
  3585  9316  9137  5395  9658  8050  4044  3187  5650] iterations.
0.3877568347767023 0.3597384955000849
Trained [247 334 266 218 157 299 184] iterations.
Trained [ 5207 11359 12661  2845 11918  9721  3256  6793  6029  6475  3231 10162
  3515  8194  9803  5629  9530  8715  4526  3431  5923] iterations.
0.4616886518405638

/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda

0.3868755655031192 0.2856802704890709
Trained [239 191 190 262 223 321 236] iterations.
Trained [ 4378  9202 11261  2513 11850  9115  2634  5596  5669  5533  2458  8680
  3048  7472  8949  4885  9550  8103  4259  3084  6348] iterations.
0.44966825416208733 0.5035209834330606
Trained [287 232 220 246 218 278 197] iterations.
Trained [ 4657 10144 12603  1665 11791 10835  2871  5194  4485  6440  2394  9551
  3221  7904  9323  2589  9974  9115  3485  2026  7625] iterations.
0.3451281074388037 0.32426897799790455
Trained [227 221 190 249 178 262 214] iterations.
Trained [ 4909 10585 12887  2944 13064 15226  2768  6173  6799  6169  2612 10115
  3421  8040  9587  4949 10198  9566  5025  3362  7041] iterations.
0.3529715495430371 0.2983941756596272
Trained [231 237 188 226 222 255 267] iterations.
Trained [ 3749 10620 10748  2794  9620  9539  3029  5573  5367  5928  1643 10448
  3798  8850  9074  5662  9500  8014  4076  3233  6017] iterations.
0.38550687722873156 0.36147902869757176
Trained [2

/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda

...

0.38825658364684035 0.28520405733606363
Trained [246 225 227 173 176 309 197] iterations.
Trained [ 4210  9486 10821  2576 11389  9873  2685  5876  5124  5879  2396  8791
  3247  7139  9021  4665  9096  7976  4312  3221  6115] iterations.
0.45052305938861076 0.5013636178613587
Trained [217 186 255 270 188 299 229] iterations.
Trained [ 4556 10265 13443  1581 12369 10880  2897  4725  4669  6123  2334  9591
  3306  7760  9489  2756 10483  8947  3510  1948  7714] iterations.
0.3460805791027717 0.32288789408515095
Trained [269 219 155 245 226 302 220] iterations.
Trained [ 4771 10490 12924  2932 13383 14180  2862  6351  6531  5996  2764  9853
  3216  8187  9587  5054 10171  9869  4954  3316  7497] iterations.
0.35059637527753396 0.29854907832911654
Trained [212 189 168 252 277 214 205] iterations.
Trained [ 3584 10848 10881  2490  9853  9465  3003  5974  5523  5798  1656 10449
  3556  8550  8926  5409  9423  8109  4159  3262  5637] iterations.
0.3879690949227373 0.3633044659534726
Trained 

/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda

0.3885423115386447 0.2856802704890709
Trained [241 247 189 218 237 273 215] iterations.
Trained [ 4290  9497 11121  2645 11890  9272  2776  5755  5448  5636  2602  8974
  3218  7476  9067  5114 10169  8304  4184  3162  6329] iterations.
0.4487769139973137 0.5029101713541455
Trained [259 269 227 204 148 300 222] iterations.
Trained [ 4834 10496 12554  1621 12213 10997  2974  4785  4414  6244  2413  9827
  3264  8395  9556  2707 10446  9273  3423  1916  7411] iterations.
0.3428897990284789 0.3215544337555958
Trained [342 235 205 230 213 306 182] iterations.
Trained [ 4948 10522 12862  2908 13181 14971  2802  6681  5895  6053  2579  9795
  3646  8081  9485  5286 10246  9558  4879  3340  7058] iterations.
0.35214540197242733 0.29648370940259205
Trained [221 225 199 169 248 258 182] iterations.
Trained [ 3852 10599 10716  2420 10033  9300  3076  5847  5676  5634  1780 11062
  3885  8895  8883  5621  9314  8024  4233  3269  5531] iterations.
0.3888181355068772 0.36258278145695366
Trained [20

/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuxiawang/miniconda

0.387970855755036 0.28753750178579934
Trained [263 256 123 258 189 202 248] iterations.
Trained [ 4254  9332 11589  2673 10830  9707  2688  5535  5028  5537  2745  8976
  3146  7112  8864  4907  9389  8577  4208  3145  6107] iterations.
0.45308747506818087 0.5057597590263362


## Print Table results

### Subtask A

- given a model setting, 
- extract v["macro avg"]["precision"], v["macro avg"]["recall"], v["macro avg"]["f1-score"], v["accuracy"]
- calculate the mean and std across five random seeds

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_json("./test_results/gltr_subtaskA.json")
df = df.T
# print(set(list(df["model"])))

models = ['no', 'davinci', 'chatGPT', 'gpt4', 'cohere', 'dolly', 'bloomz']
random_seeds = [0, 10, 30, 55, 75]

# data = df[df["model"]=='no']
table = {}
for model in models:
    data = df[df["model"]==model]

    field = ["1", "macro avg", "0"][0]
    prec = [v[field]["precision"] for v in data["results"]]
    recall = [v[field]["recall"] for v in data["results"]]
    f1 = [v[field]["f1-score"] for v in data["results"]]
    acc = [v["accuracy"] for v in data["results"]]
    metrics = {"prec": prec, "recall": recall, "f1-score": f1, "accuracy": acc}
    temp = {}
    for k, r in metrics.items():
        temp[k] = round(np.mean(r)*100,2)
    table[model] = temp

table = pd.DataFrame(table).T
# table
print(table.round(2).astype(str).to_latex())

### Subtask B

- given a domain, and a classifier
- extract acc, prec, recall, macro-F1
- extract each generator acc
- calculate the mean over five seeds

In [2]:
import pandas as pd
import numpy as np
df = pd.read_json("./test_results/subtaskB_0604.json")
# df = pd.read_json("./test_results/subtaskB_with_outfox.json")
df = df.T
# print(set(list(df['domain'])))
# df[df["domain"]=="no"]

id2label = {0: 'chatGPT', 1: 'human', 2: 'cohere', 3: 'davinci', 4: 'bloomz', 5: 'dolly', 6: 'gpt4'}
ids = [1, 3, 0, 6, 2, 5, 4]
domains = ['no', 'arxiv', 'peerread', 'reddit', 'wikihow', 'wikipedia', 'outfox']

for classifier in ["results_ovr", "results_ovo"]:
    table = {}
    for domain in domains:
        data = df[df["domain"]==domain]
        prec = [v["macro avg"]["precision"] for v in data[classifier]]
        recall = [v["macro avg"]["recall"] for v in data[classifier]]
        f1 = [v["macro avg"]["f1-score"] for v in data[classifier]]
        acc = [v["accuracy"] for v in data[classifier]]
        metrics = {"prec": prec, "recall": recall, "f1-score": f1, "accuracy": acc}

        # each generator f1-score
        for generator_index in ids:
            metrics[id2label[generator_index]] = [v[str(generator_index)]["f1-score"] for v in data[classifier]]
        
        temp = {}
        for k, r in metrics.items():
            temp[k] = round(np.mean(r)*100,2)
        table[domain] = temp

    table = pd.DataFrame(table).T
    # table
    print(table.round(2).astype(str).to_latex())
    table.to_latex(f"{classifier}_subtaskB.tex")
    print("\n\n")

\begin{tabular}{llllllllllll}
\toprule
 & prec & recall & f1-score & accuracy & human & davinci & chatGPT & gpt4 & cohere & dolly & bloomz \\
\midrule
no & 42.36 & 43.96 & 40.32 & 45.06 & 65.13 & 11.98 & 40.22 & 14.39 & 44.88 & 33.0 & 72.65 \\
arxiv & 26.24 & 34.45 & 26.92 & 34.45 & 58.38 & 0.06 & 17.78 & 0.34 & 37.59 & 35.84 & 38.47 \\
peerread & 42.2 & 44.1 & 39.04 & 44.32 & 60.13 & 0.49 & 48.37 & 19.08 & 37.83 & 40.61 & 66.77 \\
reddit & 45.54 & 46.28 & 41.7 & 46.28 & 85.06 & 24.08 & 22.44 & 19.77 & 55.74 & 14.6 & 70.23 \\
wikihow & 41.86 & 39.39 & 38.24 & 38.74 & 59.14 & 13.92 & 32.06 & 1.03 & 57.84 & 19.52 & 84.16 \\
wikipedia & 41.62 & 36.95 & 34.62 & 35.18 & 67.62 & 8.15 & 16.62 & 0.7 & 33.41 & 35.59 & 80.25 \\
outfox & 29.68 & 32.38 & 29.18 & 38.78 & 49.79 & 12.24 & 37.5 & 0.0 & 1.71 & 17.51 & 85.49 \\
\bottomrule
\end{tabular}




\begin{tabular}{llllllllllll}
\toprule
 & prec & recall & f1-score & accuracy & human & davinci & chatGPT & gpt4 & cohere & dolly & bloomz \\
\midru